In [1]:
import numpy as np
from datetime import datetime
from ann_structure import *
from gradient_checking import dict_to_vector, approximate_gradient, gradient_difference

%load_ext autoreload
%autoreload 2

# MLP Implementation:

In [2]:
def L_layer_model(X, Y, layer_dims, learning_rate =0.05, num_iterations=60, print_cost=True, grad_check=False)
    
    costs = []
    
    parameters = init_params(layer_dims)
    
    for i in range(num_iterations):
    
        # forward pass:
        AL, caches = L_forward(X, parameters)

        # calculate the cost:
        #S = softmax(AL)
        cost = cost_total(AL, Y)
        costs.append(cost)

        # backward pass:
        grads = L_backward(AL, Y, caches)
            
        # (optional) gradient checking:
        if grad_check:
            grad_approx = approximate_gradient(parameters, X, Y, epsilon=1e-7)
            grad_exact, _ = dict_to_vector(grads)
            _ = gradient_difference(grad_approx, grad_exact)
            
        
        # update parameters:
        parameters = update_parameters(parameters, grads, learning_rate)
        
        # (optional) print the cost:
        if print_cost and any([(i+1)%10==0, i == 0]):
            print('Cost after ' + str(i + 1) + ' iterations is ' + str(cost) + ' at ' + str(datetime.now()))
    
    return parameters

## Helper Functions:

In [3]:
def predict(X, parameters):
    
    L = len(parameters) // 2
    m = X.shape[1]
    
    AL, caches = L_forward(X, parameters)
    
    S = AL.copy()
    max_prob = np.argmax(S, axis=0)
        
    for i in range(m):
        S[:, i] = 0
        S[max_prob[i], i] = 1
    
    return S


def accuracy(Y_hat, Y):
    
    assert(Y_hat.shape == Y.shape)
    
    m = Y.shape[1] # number of data entries
    
    w = 0
    c = 0
    
    A = Y_hat.T
    B = Y.T
    assert(A.shape == B.shape)
    for i in range(Y.shape[1]):
        if np.all(A[i] == B[i]):
            c += 1
        else:
            w += 1
    
    return c/m

In [6]:
def standard_score_rescaling(X):
    '''
    Rescales the features to their Z-score
    '''
    m = X.shape[1]
    mu = (1 / m) * np.sum(X, axis=1, keepdims=True)
    
    X = X - mu
    
    sigma = X.var(axis=1, keepdims=True)

    X = X / np.sqrt(sigma)
    
    return X

def feature_rescaling(X):
    '''
    Rescales the features to the interval [0, 1]
    '''
    mn = np.min(X, axis=1, keepdims=True)
    mx = np.max(X, axis=1, keepdims=True)
    
    X = (X - mn) / (mx - mn)
    
    return X

# Train and Test:

In [5]:
X = np.load('data/covtype_train_x.npy')
Y = np.load('data/covtype_train_y.npy')

In [10]:
X = feature_rescaling(X)
layer_dims = (54, 70, 50, 30, 20, 7)

In [16]:
params = L_layer_model(X, Y, layer_dims, learning_rate =0.05, num_iterations=40)

Cost after 1 iterations is 1.94591006325 at 2017-10-03 02:05:13.166365
Cost after 10 iterations is 1.85873125848 at 2017-10-03 02:05:48.674808
Cost after 20 iterations is 1.770083581 at 2017-10-03 02:06:29.594596
Cost after 30 iterations is 1.69004439826 at 2017-10-03 02:07:03.975002
Cost after 40 iterations is 1.61841004275 at 2017-10-03 02:07:37.443073


In [77]:
out = predict(X, params)
out = out.squeeze()
accuracy(out, Y)